In [1]:
import tensorflow as tf
import input
import numpy as np
import matplotlib.pyplot as plt
import datetime

FLAGS = tf.app.flags.FLAGS

if tf.gfile.Exists(FLAGS.checkpoint_dir) == False:
    tf.gfile.MakeDirs(FLAGS.checkpoint_dir)

In [2]:
keep_prob = tf.placeholder(tf.float32)
images, labels = input.get_data('train', FLAGS.batch_size)

In [3]:
def weight_variable(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.1)
    var = tf.Variable(initial, name=name)
    return var

def bias_variable(shape, name):
    initial = tf.constant(0.1, shape=shape)
    var = tf.Variable(initial, name=name)
    return var

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool(x, height, width):
    return tf.nn.max_pool(x, ksize=[1, height, width, 1], strides=[1, height, width, 1], padding='SAME')



In [4]:
num_conv = FLAGS.num_conv
kernel_size = FLAGS.kernel_size
pool_size = FLAGS.pool_size
num_map = FLAGS.num_map
num_fc_layer = FLAGS.num_fc_layer
num_fc_input = FLAGS.num_fc_input

height = FLAGS.height
width = FLAGS.width
prev_num_map = FLAGS.depth
h_pool = images

for i in range(num_conv):
    W_conv = weight_variable([kernel_size, kernel_size, prev_num_map, num_map], 'W_conv' + str(i+1)) 
    #1.19x19x3x32
    #2.19x19x32x64
    b_conv = bias_variable([num_map], 'b_conv' + str(i+1))
    h_conv = tf.nn.relu(conv2d(h_pool, W_conv) + b_conv)
    h_pool = max_pool(h_conv, pool_size, pool_size)
    #1. 
    prev_num_map = num_map
    num_map = int(round(num_map*2))
    height = int(round(height/2))
    width = int(round(width/2))
    print(height,width,num_map)
    print(h_pool)

num_map = (int)(num_map/2)
print(h_pool, height*width*num_map)


32 32 64
Tensor("MaxPool:0", shape=(10, 32, 32, 32), dtype=float32)
16 16 128
Tensor("MaxPool_1:0", shape=(10, 16, 16, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(10, 16, 16, 64), dtype=float32) 16384


In [5]:
h_fc_input = tf.reshape(h_pool, [-1, height * width * num_map])
prev_num_fc_input = height * width * num_map

In [6]:
for i in range(num_fc_layer):
    W_fc = weight_variable([prev_num_fc_input, num_fc_input], 'W_fc' + str(i+1))
    b_fc = bias_variable([num_fc_input], 'b_fc' + str(i+1))
    h_fc = tf.nn.relu(tf.matmul(h_fc_input, W_fc) + b_fc)
    h_fc_input = tf.nn.dropout(h_fc, keep_prob)
    prev_num_fc_input = num_fc_input
    num_fc_input = (int)(num_fc_input/2)

In [7]:
num_fc_input = (int)(num_fc_input*2)
W_fc = weight_variable([num_fc_input, FLAGS.num_class], 'W_fc' + str(i+2))
b_fc = bias_variable([FLAGS.num_class], 'b_fc' + str(i+2))

hypothesis = tf.matmul(h_fc_input, W_fc) + b_fc
cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=hypothesis, labels=labels))
train_step = tf.train.AdamOptimizer(FLAGS.learning_rate).minimize(cross_entropy)

cost_sum = tf.summary.scalar("cost", cross_entropy)

with tf.Session() as sess:

    saver = tf.train.Saver()
    if tf.gfile.Exists(FLAGS.checkpoint_dir + '/test'):
        saver.restore(sess, FLAGS.checkpoint_dir + '/test')
    else:
        init = tf.global_variables_initializer()
        sess.run(init)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    
    writer = tf.summary.FileWriter("./logs/cost_log_test",sess.graph)
    merge_sum = tf.summary.merge_all()

    for step in range(FLAGS.max_steps):
        summary, _ = sess.run([merge_sum,train_step], feed_dict={keep_prob: 0.7})
        writer.add_summary(summary, step)
        conv1_vals, cutoff1_vals = sess.run([h_conv, h_pool], feed_dict={keep_prob: 1.0})
        fig = plt.figure(figsize=(16,16))
        for f in range():
            subplot = fig.add_subplot(16, 16, f+1)
            subplot.set_xticks([])
            subplot.set_yticks([])
            subplot.imshow(conv1_vals[0,:,:,f], cmap=plt.cm.gray_r, interpolation='nearest') 
            plt.show()
        print(step, sess.run(cross_entropy, feed_dict={keep_prob: 1.0}))
        if step % 100 == 0 or (step + 1) == FLAGS.max_steps:
            saver.save(sess, FLAGS.checkpoint_dir + '/test')
    coord.request_stop()
    coord.join(threads)


SyntaxError: invalid syntax (<ipython-input-7-7920c9a27232>, line 29)